In [80]:
import cv2
from win32api import GetSystemMetrics
import numpy as np
import pandas as pd
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

def resizeImage(image):
    (h, w) = image.shape[:2]
    x = GetSystemMetrics(0) / w
    y = GetSystemMetrics(1) / h
    if x<y:
        image = cv2.resize(image, (0,0), fx=x, fy=x)
    else:
        image = cv2.resize(image, (0,0), fx=y, fy=y)
    return image

image = imread("03_connector_2-02/본딩 - 유무/1. 원본.bmp")

img_h = image.shape[0]
img_w = image.shape[1]

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.rectangle(gray, (int(img_w/2+50), 0), (int(img_w), int(img_h)), (255,255,255),-1 )
ret, binary = cv2.threshold(gray, 145, 255, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
binary2 = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations = 2)
contours, hierarchy = cv2.findContours(binary2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

c = max(contours, key=cv2.contourArea)

hull = cv2.convexHull(c, returnPoints=False)
defects = cv2.convexityDefects(c, hull)

points = []
for i in range(defects.shape[0]):
    s, e, f, d = defects[i,0]
    start = tuple(c[s][0])
    end = tuple(c[e][0])
    far = tuple(c[f][0])

    points.append(list(c[f][0]))
    # cv2.line(image, start, end, [0,255,0], 2)
    # cv2.circle(image, far, 3, [255,0,0], 2)

pointsDF = pd.DataFrame(points, columns=['x','y'])
pointsDF= pointsDF.sort_values(by='y')
pointsDF_up = pointsDF[pointsDF.y < pointsDF.y.iloc[0]+30]
pointsDF= pointsDF.sort_values(by='y', ascending=False)
pointsDF_down = pointsDF[pointsDF.y > pointsDF.y.iloc[0]-30]

pointsDF_up_left = pointsDF_up[pointsDF_up.x < pointsDF_up.x.mean()]
pointsDF_up_right = pointsDF_up[pointsDF_up.x > pointsDF_up.x.mean()]
pointsDF_down_left = pointsDF_down[pointsDF_down.x < pointsDF_down.x.mean()]
pointsDF_down_right = pointsDF_down[pointsDF_down.x > pointsDF_down.x.mean()]

points = [ [pointsDF_up_left.x.min(), pointsDF_up_left.y.min()],
        [pointsDF_up_right.x.max(), pointsDF_up_right.y.min()], 
        [pointsDF_down_left.x.min(), pointsDF_down_left.y.max()], 
        [pointsDF_down_right.x.max(), pointsDF_down_right.y.max()] ]

cv2.line(image, tuple(points[0]), tuple(points[1]), (0,255,255), 2)
cv2.line(image, tuple(points[1]), tuple(points[3]), (0,255,255), 2)
cv2.line(image, tuple(points[3]), tuple(points[2]), (0,255,255), 2)
cv2.line(image, tuple(points[2]), tuple(points[0]), (0,255,255), 2)

centerLine = [ [int((points[0][0] + points[1][0])/2), int((points[0][1] + points[1][1])/2)], 
            [int((points[2][0] + points[3][0])/2), int((points[2][1] + points[3][1])/2)] ]

# cv2.line(image, tuple(centerLine[0]), tuple(centerLine[1]), (0,255,255), 2)

def dist(P, A=centerLine[0], B=centerLine[1]):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

def getMoments(contour):
    M = cv2.moments(cnt)
    x = int(M['m10'] / (M['m00'] + 1e-5))
    y = int(M['m01'] / (M['m00'] + 1e-5))
    return [x, y]

momentsList = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 300 and area < 2200:
        p = getMoments(cnt)
        d = dist(p)
        cv2.drawContours(image, [cnt], 0, (0,255, 0), 2)
        cv2.circle(image, tuple(p), 2, (255, 0, 0), -1)
        if d <100:
            momentsList.append(p)

momentsListDF = pd.DataFrame(momentsList, columns=['x','y'])
momentsLeft = momentsListDF[momentsListDF.x < momentsListDF.x.mean()].sort_values(by='y')
momentsRight = momentsListDF[momentsListDF.x > momentsListDF.x.mean()].sort_values(by='y')

cv2.line(image, tuple(momentsLeft.iloc[0]), tuple(momentsLeft.iloc[-1]), (255,255,0), 2)
cv2.line(image, tuple(momentsLeft.iloc[-1]), tuple(momentsRight.iloc[-1]), (255,255,0), 2)
cv2.line(image, tuple(momentsRight.iloc[-1]), tuple(momentsRight.iloc[0]), (255,255,0), 2)
cv2.line(image, tuple(momentsRight.iloc[0]), tuple(momentsLeft.iloc[0]), (255,255,0), 2)

for i in range(len(momentsLeft)):
    cv2.putText(image, str(i+1), (momentsLeft.iloc[i,0]+20,momentsLeft.iloc[i,1]+10),cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0), 2)
for i in range(len(momentsRight)):
    cv2.putText(image, str(i+1+len(momentsLeft)), (momentsRight.iloc[i,0]-60,momentsRight.iloc[i,1]+10),cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0), 2)

image = resizeImage(image)
cv2.imshow("img", image)
# cv2.imshow("gray2", gray)
# cv2.imshow("binary", binary2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Width = 1920
Height = 1080


In [15]:
import cv2
from win32api import GetSystemMetrics
import numpy as np
import pandas as pd
print("Width =", GetSystemMetrics(0))
print("Height =", GetSystemMetrics(1))

# 파일명 한글 허용 함수
def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None

# 이미지 크기 조절 함수
def resizeImage(image):
    (h, w) = image.shape[:2]
    x = GetSystemMetrics(0) / w
    y = GetSystemMetrics(1) / h
    if x<y:
        image = cv2.resize(image, (0,0), fx=x, fy=x)
    else:
        image = cv2.resize(image, (0,0), fx=y, fy=y)
    return image

image = imread("03_connector_2-02/본딩 - 유무/1. 원본.bmp")

img_h = image.shape[0]
img_w = image.shape[1]

gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
gray = cv2.rectangle(gray, (int(img_w/2+50), 0), (int(img_w), int(img_h)), (255,255,255),-1 )
ret, binary = cv2.threshold(gray, 145, 255, cv2.THRESH_BINARY_INV)
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
binary2 = cv2.morphologyEx(binary, cv2.MORPH_OPEN, kernel, iterations = 2)
contours, hierarchy = cv2.findContours(binary2, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)

c = max(contours, key=cv2.contourArea) # 최대크기 컨투어
maxContourDF = c.reshape(len(c), 2)
maxContourDF = pd.DataFrame(maxContourDF, columns=['x','y']).sort_values(by='y') # 최대크기 컨투어 좌표 DF화 및 y값으로 정렬

pointsUp = maxContourDF[maxContourDF.y < maxContourDF.y.iloc[0]+30] # 상단 컨투어
pointsDown = maxContourDF[maxContourDF.y > maxContourDF.y.iloc[-1]-30] # 하단 컨투어

pointsUpLeft = pointsUp[pointsUp.x < pointsUp.x.mean()] # 좌상단
pointsUpRight = pointsUp[pointsUp.x > pointsUp.x.mean()] # 우상단
pointsDownLeft = pointsDown[pointsDown.x < pointsDown.x.mean()] # 좌하단
pointsDownRight = pointsDown[pointsDown.x > pointsDown.x.mean()] # 우하단

# 모서리
points = [ [pointsUpLeft.x.min(), int(pointsUpLeft.y.mean())],
        [pointsUpRight.x.max(), int(pointsUpRight.y.mean())], 
        [pointsDownRight.x.max(), int(pointsDownRight.y.mean())],
        [pointsDownLeft.x.min(), int(pointsDownLeft.y.mean())] ]

# 사각형 그리기
cv2.polylines(image,[np.array(points)], True, (0,255,255), 2)

# 사각형의 센터라인 양 끝 좌표
centerLine = [ [int((points[0][0] + points[1][0])/2), int((points[0][1] + points[1][1])/2)], 
            [int((points[2][0] + points[3][0])/2), int((points[2][1] + points[3][1])/2)] ]
# 센터라인 그리기
# cv2.line(image, tuple(centerLine[0]), tuple(centerLine[1]), (0,255,255), 2)

# 선과 점 사이의 최소 거리 구하는 함수
def dist(P, A=centerLine[0], B=centerLine[1]):
    area = abs ( (A[0] - P[0]) * (B[1] - P[1]) - (A[1] - P[1]) * (B[0] - P[0]) )
    AB = ( (A[0] - B[0]) ** 2 + (A[1] - B[1]) ** 2 ) ** 0.5
    return ( area / AB )

# 모멘트 함수
def getMoments(contour):
    M = cv2.moments(cnt)
    x = int(M['m10'] / (M['m00'] + 1e-5))
    y = int(M['m01'] / (M['m00'] + 1e-5))
    return [x, y]

momentsList = []
for cnt in contours:
    area = cv2.contourArea(cnt)
    if area > 300 and area < 2200: # 컨투어의 크기 비교
        p = getMoments(cnt)
        d = dist(p)
        cv2.drawContours(image, [cnt], 0, (0,255, 0), 2)
        cv2.circle(image, tuple(p), 2, (255, 0, 0), -1)
        if d <100: # 모멘트와 센터라인의 거리 비교
            momentsList.append(p)

momentsListDF = pd.DataFrame(momentsList, columns=['x','y'])
momentsLeft = momentsListDF[momentsListDF.x < momentsListDF.x.mean()].sort_values(by='y') # 좌측 모멘트
momentsRight = momentsListDF[momentsListDF.x > momentsListDF.x.mean()].sort_values(by='y') # 우측 모멘트

# 모멘트 사각형 모서리 좌표
momentsPoints = [ list(momentsLeft.iloc[0]),
                list(momentsLeft.iloc[-1]),
                list(momentsRight.iloc[-1]),
                list(momentsRight.iloc[0]) ]

# 모멘트 사각형 그리기
cv2.polylines(image,[np.array(momentsPoints)], True, (255,255,0), 2)

# 좌측 모멘트 넘버링
for i in range(len(momentsLeft)):
    cv2.putText(image, str(i+1), (momentsLeft.iloc[i,0]+20,momentsLeft.iloc[i,1]+10),cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0), 2)
# 우측 모멘트 넘버링
for i in range(len(momentsRight)):
    cv2.putText(image, str(i+1+len(momentsLeft)), (momentsRight.iloc[i,0]-60,momentsRight.iloc[i,1]+10),cv2.FONT_HERSHEY_COMPLEX, 1, (0,255, 0), 2)

image = resizeImage(image)
cv2.imshow("img", image)
# cv2.imshow("gray2", gray)
# cv2.imshow("binary", binary2)
cv2.waitKey(0)
cv2.destroyAllWindows()

Width = 1920
Height = 1080


In [ ]:
import cv2
import numpy as np
from win32api import GetSystemMetrics


def imread(filename, flags=cv2.IMREAD_COLOR, dtype=np.uint8):
    try:
        n = np.fromfile(filename, dtype)
        img = cv2.imdecode(n, flags)
        return img
    except Exception as e:
        print(e)
        return None


def FitToWindowSize(image):
    image_resized = image.copy()
    # 윈도우 크기 얻기
    # from win32api import GetSystemMetrics
    # print("Width =", GetSystemMetrics(0))
    # print("Height =", GetSystemMetrics(1))
    # 이미지 크기 얻기
    print('image {}'.format(image.shape))
    win_w = GetSystemMetrics(0)
    win_h = GetSystemMetrics(1)
    img_h, img_w = image.shape[:2]

    if(img_h > win_h or img_w > win_w):
        rate_width = (win_w / img_w)*0.95
        rate_height = (win_h / img_h)*0.95
        scale = rate_width if (rate_width < rate_height) else rate_height
        image_resized = cv2.resize(image, dsize=(
            0, 0), fx=scale, fy=scale, interpolation=cv2.INTER_LINEAR)
    # cv2.imshow('image_resize',image_resized)
    return image_resized

class D1221():
    def __init__(self):
        self.center =[]
        self.Big = []
        self.maxwh = 0
        self.RECT=[]
        self.MMT = []
        self.RECT2 = []
        self.ANY = []
    # 큰 사각형 따기    
    def drawconT(self, inputimg, image):

        self.gray = cv2.cvtColor(inputimg, cv2.COLOR_BGR2GRAY)
        ret, binary = cv2.threshold(self.gray,90, 255, cv2.THRESH_BINARY_INV)
        edges = cv2.Canny(binary,100,200,apertureSize = 3)

        # 코너 따기
        corners = cv2.goodFeaturesToTrack(edges, 3000, 0.05, 1, blockSize=3, useHarrisDetector=True, k=0.03)
        for i in corners:
            self.RECT.append(list(i[0]))
        self.RECT.sort(key = lambda REC : REC[0])
        # 먼저 왼쪽 두 좌표 따기
        self.left = self.RECT[:20].copy()
        self.left.sort(key = lambda REC : REC[1])
        self.RECT.clear()
        self.RECT.append(self.left[0])
        self.RECT.append(self.left[-1])
        # 오른 쪽 두 좌표 따기
        for i in corners:
            if np.abs(i[0][0]-self.RECT[0][0]) <image.shape[1]/2:
                
                self.RECT2.append(list(i[0]))
        self.RECT2.sort(key = lambda REC :REC[1])
        upRight = self.RECT2[:30]
        downRight = self.RECT2[-30:]
        upRight.sort(key = lambda up : up[0])
        downRight.sort(key = lambda down : down[0])
        self.RECT.append(upRight[-1])
        self.RECT.append(downRight[-1])
        self.RECT2.clear()

       
        for i in corners:
            if i[0][0]-self.RECT[2][0] >0 and  i[0][0]-self.RECT[2][0]<100 and abs(i[0][1] - self.RECT[2][1])>50 and abs(i[0][1] - self.RECT[3][1])>50 :
                self.ANY.append(list(i[0]))
        self.ANY.sort(key = lambda REC : REC[1])
        self.RECT2.append(self.ANY[0])
        self.RECT2.append(self.ANY[-1])
        cv2.circle(image,tuple(self.RECT2[0]),3,(255,0,0))
        cv2.circle(image,tuple(self.RECT2[1]),3,(255,0,0))
        self.ANY.clear()

        
        image2 = image[:,int(self.RECT[2][0]):].copy()
        
        gray2 =  cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)
        _, binary2 = cv2.threshold(gray2,36, 255, cv2.THRESH_BINARY_INV)
        corners = cv2.goodFeaturesToTrack(binary2, 3000, 0.05, 1, blockSize=3, useHarrisDetector=True, k=0.03)
        for i in corners:
            self.ANY.append(list(i[0]))

        self.ANY.sort(key= lambda ANY : ANY[0])
        ANY2 = self.ANY[int(len(self.ANY)/2):]
        ANY2.sort(key = lambda ANY2 : ANY2[1])
        X  = ANY2[0][0]
        image2 = image2[:,:int(X)].copy()
        gray2 =  cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY) 
        count = 0
        ret, binary2 = cv2.threshold(gray2,100, 255, cv2.THRESH_BINARY_INV)

        #오른쪽 contour
        kernel = np.ones((3,3),np.uint8)
        binary3 = cv2.erode(binary2,kernel,iterations=1)

        contours, __ = cv2.findContours(
            binary3, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
        for i in range(len(contours)):
            cv2.drawContours(image2,contours,i,(255,0,255),1)
        

        ret, binary2 = cv2.threshold(gray2,60, 255, cv2.THRESH_BINARY_INV)
        contours, __ = cv2.findContours(
            binary2, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        for i in range(len(contours)):
            x, y, w, h = cv2.boundingRect(contours[i])
            M = cv2.moments(contours[i])
            if w*h < 50000 and w*h> 1000:
                count+=1

        if count == 17 :
            for i in range(len(contours)):
                x, y, w, h = cv2.boundingRect(contours[i])
                if w*h < 50000 and w*h> 1000:
                    cv2.rectangle(image2,(x,y),(x+w,y+h),(255,0,0),2)
        


        image[:,int(self.RECT[2][0]):int(self.RECT[2][0])+int(X)] = image2
        # 사각형 그리기
        cv2.line(image, tuple(self.RECT[0]),tuple(self.RECT[1]),(255,0,0))
        cv2.line(image, tuple(self.RECT[1]),tuple(self.RECT[3]),(255,0,0))   
        cv2.line(image, tuple(self.RECT[2]),tuple(self.RECT[3]),(255,0,0))   
        cv2.line(image, tuple(self.RECT[0]),tuple(self.RECT[2]),(255,0,0)) 
        # 중심점
        self.MMT.append(int(np.mean(self.RECT,axis=0)[0]))
        self.MMT.append(int(np.mean(self.RECT,axis=0)[1]))
        cv2.circle(image,tuple(self.MMT),2,(255,255,0),0)
        
        image = self.CenterCon(inputimg,image)
        if count != 17:
                print(count)
                cv2.putText(image, "Right Error", (800,100),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))

        return image

    #작은 사각형 따기
    def CenterCon(self,inputimg,image):
        self.gray = cv2.cvtColor(inputimg, cv2.COLOR_BGR2GRAY)
        self.sideCon = []
        _, binary = cv2.threshold(self.gray, 125, 255, cv2.THRESH_TOZERO)
        contours, __ = cv2.findContours(
            binary, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE)
        for i in range(len(contours)):
            if cv2.contourArea(contours[i]) < 550 and cv2.contourArea(contours[i]) > 100:
                x, y, w, h = cv2.boundingRect(contours[i])
                M = cv2.moments(contours[i])
                cX = int(M['m10'] / M['m00']+ 1e-5)
                cY = int(M['m01'] / M['m00']+ 1e-5)
                if np.abs(cX-self.MMT[0])<60:
                    cv2.drawContours(image, contours,i,(255,255,0))
                    self.center.append([cX,cY])
                elif np.abs(cX-self.MMT[0])>60 and np.abs(cX-self.MMT[0])<90:
                    cv2.drawContours(image, contours,i,(0,255,255))
                    self.sideCon.append([cX,cY])
        
        if len(self.center) != 38 :
            cv2.putText(image, "center RECT error", (100,100),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
            print(len(self.center))
        else:
            self.center.sort(reverse=True)
            self.rightside = self.center[:19]
            self.leftside = self.center[19:]
            for i in self.rightside:
                if i[0]<self.MMT[0]:
                    cv2.putText(image, "center RECT error", (100,100),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
            for i in self.leftside:
                if i[0]>self.MMT[0]:
                    cv2.putText(image, "center RECT error", (100,100),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255))
            self.rightside.sort(key= lambda side : side[1])
            self.leftside.sort(key= lambda side : side[1])
            for i in range(len(self.leftside)):
                cv2.putText(image,str(i+18),(self.rightside[i][0]-30,self.rightside[i][1]+5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0))
                cv2.putText(image,str(i),(self.leftside[i][0]+10,self.leftside[i][1]+5),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,0))

            cv2.line(image, (self.leftside[0][0],self.leftside[0][1]),(self.leftside[-1][0],self.leftside[-1][1]),(255,255,0))
            cv2.line(image, (self.rightside[0][0],self.rightside[0][1]),(self.rightside[-1][0],self.rightside[-1][1]),(255,255,0))
            cv2.line(image, (self.leftside[0][0],self.leftside[0][1]),(self.rightside[0][0],self.rightside[0][1]),(255,255,0))
            cv2.line(image, (self.leftside[-1][0],self.leftside[-1][1]),(self.rightside[-1][0],self.rightside[-1][1]),(255,255,0))

        return image

if __name__ == '__main__':
    
    #하나만 비교

    # D2 = D1221()
    # src = imread("C:/Users/w/Desktop/OPENCV/03_connecter/5.bmp")
    # src = FitToWindowSize(src)
    # dst = src.copy()
    # output = D2.drawconT(src, dst)
    # cv2.imshow('dst', output)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

    
    # 여러개 한꺼번에 비교
    for i in range(13):
        print(i+1)
        src = imread("C:/Users/w/Desktop/OPENCV/03_connecter/"+str(i+1)+".bmp")
        D2 = D1221()
        src = FitToWindowSize(src)
        dst = src.copy()
        output = D2.drawconT(src, dst)
        cv2.imshow('dst', output)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        